# 1. Flow Dependencies (Read after Write)

Welcome to our taxonomy of data dependencies in parallel programs. Throughout this notebook, you will find several different (varyingly complex) examples of flow dependencies. We also expect to create more notebooks for other types of dependencies.

## What is a flow/RAW dependency?

A Flow dependency, also known as a "true dependency" or "read-after-write (RAW)", occurs when an instruction depends directly on the result of a previous instruction. Imagine the following sequential code:

```
1. A = 3
2. B = A
3. C = B
```
Instruction 2 relies on the output of instruction 1, and instruction 3 relies on the output of instruction 2. Instruction 2 contains a "Write after Read," as `B` is written to after the value of `A` is read. The same is true with instruction 3, as `B` must be read before `C` is written. We say these instructions are each "truly dependent" on the instruction preceding them.

Before we do any "live" examples, let's look at one more, (slightly) more complex example. Observe the following code:

```
1. x=10;
2. y=2*x+5;
```
Is there a RAW dependence here? Why?

I would posit that there is indeed a RAW dependence here. In instruction 1, `x` is written to, and immediately afterwards, `y` is written to, by reading from `x`. We can therefore say that instruction 2 depends on instruction 1 with a flow (RAW) dependence.


Now, let's look at some more examples.

## Example 1. OpenMP RAW

Consider the following loop:

```
double v=start;
int step = 1;

for (int i=0; i<N; i+= step){
    v += 2;
	v=v +step; 
    
}

```
Assume the code works. That is, assume there is a function `f` which is dependent only on v and assume that there's an `int main` and all of that.

Is there a flow dependence here? 

I would say that there is indeed a flow dependence here. Between the first and second instructions in the loop, the second instruction cannot execute until the instruction before it has. This example can still benefit from loop-level parallelsim, because each iteration of the loop will not have a problem running in parallel. To observe this, let's compile it with OpenMP.

A working version of this code is saved in `omp_1_serial.c`

Just to make sure the code compiles, we're going to compile the serial version first.

In [3]:
!gcc omp_1_serial.c -std=c99

Now, let's examine a parallel version. See if you can parallelize it using OpenMP in the cell below. As a hint, the way we can accomplish this is with a reduction, assuming the final value we care about is the value of `v` after all of the code has been run.

```
double v=start;
int step = 1;

# pragma omp parallel for reduction(+:v)
for (int i=0; i<N; i+= step){
    v += 2;
    v=v +step; 

}

```

Now, let's compile this version. It's saved in `omp_1_parallel.c`. Feel free to mess around with it.

In [5]:
!gcc -fopenmp omp_1_parallel.c -std=c99

The compiler seems fairly happy with that, and in this case, we are too!

# Example 2. OpenACC RAW

Now, we want to look at very similar code, but in this case, we want to parallelize it with OpenACC. We will see that the OpenACC compiler can be very helpful about data dependencies. We'll hopefully see in this example that OpenACC has no problem with loop-level parallelism in this example. Let's once again look at the serial code from above:

```
double v=start;
int step = 1;

for (int i=0; i<N; i+= step){
    v += 2;
    v=v +step; 

}
```

Can we parallelize this loop using vectors with OpenACC? I would say we can. One thing to remember is that OpenACC automatically generates reductions, so we don't need to worry about that. Try to parallelize this loop with OpenACC in the next cell.

```
double v=start;
int step = 1;

# pragma acc kernels
for (int i=0; i<N; i+= step){
    v += 2;
    v=v +step; 

}
```

Now, let's compile this version. It's saved in `acc_2_parallel.c`. Feel free to mess around with it.

In [7]:
!pgcc -acc acc_2_serial.c -Minfo=accel

main:
     11, Loop carried scalar dependence for v at line 12
         Accelerator serial kernel generated
         Generating Tesla code
         11, #pragma acc loop seq
     11, Loop carried scalar dependence for v at line 12


Now, we can see that the compiler is telling us what it's doing and what it's found. It has found our loop dependency and is unable to generate parallel code. Let's try to fix it now that we know our dependence is real.

In this case, we can actually fix it in a pretty simple way. We can actually eliminate the line of code in which there's a dependence and simply have a reduction:

```
double v=start;
int step = 1;

# pragma acc kernels
for (int i=0; i<N; i+= step){
    v = v + step +2; 

}
```

In [12]:
!pgcc -acc acc_2_parallel.c -Minfo=accel

main:
      9, Generating implicit copyout(v[:])
     10, Loop is parallelizable
         Generating Tesla code
         10, #pragma acc loop gang, vector(128) /* blockIdx.x threadIdx.x */


The loop is parallelizable!